In [ ]:
def plot_trajectory(rec_id, x_rel, y_rel, inside):
    plt.figure(figsize=(6, 6))

    # Target circle in relative coordinates
    theta = np.linspace(0, 2 * PI, 400)
    cx = TASK_RADIUS * np.cos(theta)
    cy = TASK_RADIUS * np.sin(theta)
    plt.plot(cx, cy, color="yellow", linewidth=3, label="target circle")

    # Separate inside/outside
    inside_mask  = (inside == 1)
    outside_mask = (inside == 0)

    plt.scatter(x_rel[inside_mask],  y_rel[inside_mask],  s=2, c="green", label="inside")
    plt.scatter(x_rel[outside_mask], y_rel[outside_mask], s=2, c="red",   label="outside")

    plt.axhline(0, color="lightgray", linewidth=0.5)
    plt.axvline(0, color="lightgray", linewidth=0.5)

    plt.title(f"record {rec_id} – trajectory")
    plt.xlabel("X relative (pixels)")
    plt.ylabel("Y relative (pixels)")
    plt.gca().set_aspect("equal", "box")
    plt.legend()
    plt.grid(False)
    plt.show()

In [ ]:
def plot_time_series(rec_id, t_s, x_rel, y_rel, inside):
    fig, axs = plt.subplots(3, 1, figsize=(10, 8), sharex=True)

    # X(t)
    axs[0].plot(t_s, x_rel, ".", markersize=2, label="x_rel")
    axs[0].set_ylabel("X rel (px)")
    axs[0].legend()
    axs[0].grid(True)

    # Y(t)
    axs[1].plot(t_s, y_rel, ".", markersize=2, color="orange", label="y_rel")
    axs[1].set_ylabel("Y rel (px)")
    axs[1].legend()
    axs[1].grid(True)

    # inside(t)
    axs[2].step(t_s, inside, where="post", color="green", label="inside")
    axs[2].set_ylabel("inside")
    axs[2].set_xlabel("Time (s)")
    axs[2].set_ylim(-0.1, 1.1)
    axs[2].legend()
    axs[2].grid(True)

    fig.suptitle(f"record {rec_id} — time series", y=0.98)
    plt.tight_layout()
    plt.show()